In [1]:
import pandas as pd
import csv
from datetime import datetime,date
from sigfig import round as round_sigfig
import json
import subprocess, os
from astropy import units as u
from astropy.coordinates import Angle
#basecat1_table = pd.read_csv('basecat1_results_240303.csv', index_col=0)

## WRITING CSV

In [33]:
today = datetime.today()
date_string = today.strftime("%y%m%d")
csv_name = 'basecat1_catalog.'+date_string+'.csv'
with open(csv_name, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['tns_name', 'ra', 'ra_err', 'dec', 'dec_err', 'dm','dm_err', 'fluence', 'fluence_err', 'flux', 'flux_err', 'exp_up','exp_up_err','exp_low','exp_low_err', 'notes'])
    for index, row in basecat1_table.iterrows():
        tns_name = 'FRB ' + row['tns'][3:]
        if pd.isna(row['dm_err']):
            DM = round(row['dm'], 2)
            DM_err = -9999
        else:
            DM, DM_err = round_val_err_indiv(row['dm'], row['dm_err'], 'DM')
        
        ra, ra_err = round_val_err_indiv(row['ra'], row['ra_err'], "ra")
        ra_err = int(round(Angle(row['ra_err'], u.degree).arcsec,0))
        
        dec, dec_err = round_val_err_indiv(row['dec'], row['dec_err'], "dec")
        dec_err = int(round(Angle(row['dec_err'], u.degree).arcsec,0))
        if pd.isna(row['fluence']):
            fluence = -9999
            fluence_err = -9999
            flux = -9999
            flux_err = -9999
        else:
            fluence, fluence_err = round_val_err_indiv(row['fluence'], row['fluence_err'], "fluence")    
            flux, flux_err = round_val_err_indiv(row['flux'], row['flux_err'], "flux")
        
        exp_up, exp_up_err = round_val_err_indiv(row['exposure_upper'], row['exposure_upper_err'], "exposure_upper_err")
        exp_low, exp_low_err = round_val_err_indiv(row['exposure_lower'], row['exposure_lower_err'], "exposure lower")
        if pd.isna(row['notes']):
            notes = -9999
        else:
            notes = row['notes']
        writer.writerow([tns_name, ra, ra_err, dec, dec_err, DM, DM_err, fluence, fluence_err, flux, flux_err, exp_up, exp_up_err, exp_low, exp_low_err, notes])


/Users/curtina/anaconda3/lib/python3.10/site-packages/sigfig/sigfig.py:586: UserWarning: warning: 2 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


## WRITING JSON

In [34]:
today = datetime.today()
date_string = today.strftime("%y%m%d")
all_events = []
for index, row in basecat1_table.iterrows():
    event = {}
    event["tns_name"] = 'FRB ' + row['tns'][3:]
    if pd.isna(row['dm_err']):
        event["DM"] = round(row['dm'], 2)
        event["DM_err"] = -9999
    else:
        event["DM"], event["DM_err"] = round_val_err_indiv(row['dm'], row['dm_err'], 'DM')
    event["ra"], ra_err = round_val_err_indiv(row['ra'], row['ra_err'], "ra")
    event["ra_err"] = int(round(Angle(row['ra_err'], u.degree).arcsec,0))
    event["dec"], dec_err = round_val_err_indiv(row['dec'], row['dec_err'], "dec")
    event["dec_err"] = int(round(Angle(row['dec_err'], u.degree).arcsec,0))
    if pd.isna(row['fluence']):
        event["fluence"] = -9999
        event["fluence_err"] = -9999
        event["flux"] = -9999
        event["flux_err"] = -9999
    else:
        event["fluence"], event["fluence_err"] = round_val_err_indiv(row['fluence'], row['fluence_err'], "fluence")    
        event["flux"], event["flux_err"] = round_val_err_indiv(row['flux'], row['flux_err'], "flux")
    event["exp_up"], event["exp_up_err"] = round_val_err_indiv(row['exposure_upper'], row['exposure_upper_err'], "exposure_upper_err")
    event["exp_low"], event["exp_low_err"] = round_val_err_indiv(row['exposure_lower'], row['exposure_lower_err'], "exposure lower")
    if pd.isna(row['notes']):
        event["notes"] = -9999
    else:
        event["notes"] = row['notes']

    all_events.append(event)

with open('basecat1_catalog.'+date_string+'.json', "w") as outfile:
    json.dump(all_events, outfile)
   

## WRITING LATEX FILE

In [37]:
#note margin is changed from 0.8 to 0.5 to help table 2    
header = r'''\documentclass{article}
\usepackage{longtable}
\usepackage{gensymb}
\usepackage{graphicx}
\usepackage[a4paper, margin=0.5in]{geometry}
\usepackage{scalerel}
\usepackage{threeparttablex}
\usepackage[T1]{fontenc}
\usepackage[font=normalsize]{caption}
\usepackage{etoolbox}
\appto\TPTnoteSettings{\normalsize}

%\setlength{\LTpre}{0pt}
%\setlength{\LTpost}{0pt}

\graphicspath{{./imgs/}}
\begin{document}
%\begin{landscape}
\begin{center}
'''

footer = r'''\end{center}
%\end{landscape}
\end{document}
'''

table = ''
table = table + r'''
\begin{ThreePartTable} 
\renewcommand\TPTminimum{\textwidth}
\begin{TableNotes}
  \item[a]{TNS name for the source}
  \item[b]{Exposure for upper transit of the source.}
  \item[c]{Exposure for lower transit of the source (if applicable).}
\end{TableNotes} 
\begin{longtable}[c]{c c c c c c c c c c c c c c c}
\caption{CHIME/FRB First Baseband Catalog}\endfirsthead
\caption{\textit{continued}} \endhead
\hline
\\[-2ex]
Name \tnote{a} 
& \begin{tabular}[c]{@{}c@{}}RA\\ ($\degree$)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}$\sigma_{RA}$\\ ($''$)\end{tabular}  
& \begin{tabular}[c]{@{}c@{}}Dec\\ ($\degree$)\end{tabular} 
& \begin{tabular}[c]{@{}c@{}}$\sigma_{Dec}$\\ ($''$)\end{tabular} 
& \begin{tabular}[c]{@{}c@{}}DM\\ (pc cm$^{-3}$)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Fluence\\ (Jy ms)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Flux\\ (Jy)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Up. exp. \tnote{b} \\ (Hr)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Low. exp. \tnote{c} \\ (Hr)\end{tabular}\\
\\[-2ex]
\hline
\\[-2ex]
'''
for index, row in basecat1_table.iterrows():
    tns_name = 'FRB ' + row['tns'][3:]
    # uncertainties in separate columns
    #-------ra, dec--------
    ra, ra_err = round_val_err_indiv(row['ra'], row['ra_err'], "ra")
    ra_err = int(round(Angle(row['ra_err'], u.degree).arcsec,0))
    dec, dec_err = round_val_err_indiv(row['dec'], row['dec_err'], "dec")
    dec_err = int(round(Angle(row['dec_err'], u.degree).arcsec,0))
    # uncertainties in paranthesis
    #-------DM--------
    if pd.isna(row['dm_err']):
        DM = round(row['dm'], 2)
    else:
        DM = round_sigfig(row['dm'], uncertainty = row['dm_err'], sep='brackets')
    #-------flux&fluence--------
    if pd.isna(row['fluence']):
        fluence = '--'
        flux = '--'
    else:
        fluence = round_sigfig(row['fluence'], uncertainty = row['fluence_err'], sep='brackets')
        flux = round_sigfig(row['flux'], uncertainty = row['flux_err'], sep='brackets')
        
        if row['fluence_err'] > 10:
            fluence = round_sigfig(row['fluence'], uncertainty = row['fluence_err'], sep='external_brackets')
        if row['flux_err'] > 10:
            flux = round_sigfig(row['flux'], uncertainty = row['flux_err'], sep='external_brackets')
    #-------exposure--------        
    if row['exposure_upper_err'] > 10:
        exp_up = round_sigfig(row['exposure_upper'], uncertainty =row['exposure_upper_err'], sep='external_brackets')
    else:
        exp_up = round_sigfig(row['exposure_upper'], uncertainty =row['exposure_upper_err'], sep='brackets')
        if exp_up == '1(1)0':
            exp_up = '10(9)'
           

    
    if row['exposure_lower_err'] > 10:
        exp_low = round_sigfig(row['exposure_lower'], uncertainty = row['exposure_lower_err'], sep='external_brackets')
    else:
        exp_low = round_sigfig(row['exposure_lower'], uncertainty = row['exposure_lower_err'], sep='brackets')
    
    

    if exp_low == '0(0)':
        exp_low = '--'


    if tns_name == 'FRB 20190628C':
        tns_name = r'FRB 20190628C\tablenotemark{g}'
    elif tns_name == 'FRB 20190626A':
        tns_name = r'FRB 20190626A\tablenotemark{e,m}'
    elif tns_name == 'FRB 20190625E':
        tns_name = r'FRB 20190625E\tablenotemark{m}'
    elif tns_name == 'FRB 20190621A':
        tns_name = r'FRB 20190621A\tablenotemark{g,o}'
    elif tns_name == 'FRB 20190612A':
        tns_name = r'FRB 20190612A\tablenotemark{n}'
    elif tns_name == 'FRB 20190611A':
        tns_name = r'FRB 20190611A\tablenotemark{m}'
    elif tns_name == 'FRB 20190606A':
        tns_name = r'FRB 20190606A\tablenotemark{l}'
    elif tns_name == 'FRB 20190605B':
        tns_name = r'FRB 20190606B\tablenotemark{f}'
    elif tns_name == 'FRB 20190605A':
        tns_name = r'FRB 20190606A\tablenotemark{f}'
    elif tns_name == 'FRB 20190517C':
        tns_name = r'FRB 20190517C\tablenotemark{e}'
    elif tns_name == 'FRB 20190301A':
        tns_name = r'FRB 20190301A\tablenotemark{k}'
    elif tns_name == 'FRB 20190202B':
        tns_name = r'FRB 20190202B\tablenotemark{e,j}'
    elif tns_name == 'FRB 20190117A':
        tns_name = r'FRB 20190117A\tablenotemark{i}'
    elif tns_name == 'FRB 20190116A':
        tns_name = r'FRB 20190116A\tablenotemark{h}'
    elif tns_name == 'FRB 20181231C':
        tns_name = r'FRB 20181231C\tablenotemark{g}'
    elif tns_name == 'FRB 20181228B':
        tns_name = r'FRB 20181228B\tablenotemark{e}'
    elif tns_name == 'FRB 20181225A':
        tns_name = r'FRB 20181225A\tablenotemark{f}'
    elif tns_name == 'FRB 20181226A':
        tns_name = r'FRB 20181226A\tablenotemark{f}'
    elif tns_name == 'FRB 20181220A':
        tns_name = r'FRB 20181220A\tablenotemark{e}'
        



    #-------adding to line--------
    line ='{} & {} & {} & {} & {} & {} & {} & {} & {} & {} \\\ '.format(tns_name, 
                                    ra, ra_err, dec, dec_err, DM, fluence,
                                    flux, exp_up, exp_low) 
    table += line + '\n'

table = table + r'''\hline
\\[-2ex]
\insertTableNotes
\end{longtable}
\end{ThreePartTable}
\vspace*{1cm}
'''


content = header + table + footer
print(table)




\begin{ThreePartTable} 
\renewcommand\TPTminimum{\textwidth}
\begin{TableNotes}
  \item[a]{TNS name for the source}
  \item[b]{Exposure for upper transit of the source.}
  \item[c]{Exposure for lower transit of the source (if applicable).}
\end{TableNotes} 
\begin{longtable}[c]{c c c c c c c c c c c c c c c}
\caption{CHIME/FRB First Baseband Catalog}\endfirsthead
\caption{\textit{continued}} \endhead
\hline
\\[-2ex]
Name \tnote{a} 
& \begin{tabular}[c]{@{}c@{}}RA\\ ($\degree$)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}$\sigma_{RA}$\\ ($''$)\end{tabular}  
& \begin{tabular}[c]{@{}c@{}}Dec\\ ($\degree$)\end{tabular} 
& \begin{tabular}[c]{@{}c@{}}$\sigma_{Dec}$\\ ($''$)\end{tabular} 
& \begin{tabular}[c]{@{}c@{}}DM\\ (pc cm$^{-3}$)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Fluence\\ (Jy ms)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Flux\\ (Jy)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Up. exp. \tnote{b} \\ (Hr)\end{tabular}
& \begin{tabular}[c]{@{}c@{}}Low. exp. \tnote{c} \\ (Hr)\end{tab

## Rounding algorithms

In [4]:
def round_val_err_indiv(val, err, category):
    """
    Rounds the value and error and returns each individual object
    
    Used for creating a csv file rather than a pdf.
    """

    val_rounded, err_rounded = round_sigfig(val, uncertainty=err, cutoff=99, sep=list, type=str)

    return val_rounded, err_rounded

In [3]:
def round_val_err_indiv_DRAKE(val, err, category):
    """
    Rounds the value and error and returns each individual object
    
    Used for creating a csv file rather than a pdf.
    """

    val_err = round_sigfig(val, uncertainty=err, cutoff=99, format = 'Drake', type=str)

    return val_err